In [41]:
import requests
from bs4 import BeautifulSoup
import csv
from itertools import zip_longest
import pandas as pd
def scrape_bayt_page(url, headers):
    jobs = []  # Assurez-vous que la liste jobs est définie ici
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")

        job_titles = soup.find_all("h2", {"class": "col u-stretch t-large m0 t-nowrap-d t-trim"})
        company__names = soup.find_all("div", {"class": "t-nowrap"})
        locations_names = soup.find_all("div", {"class": "t-mute t-small"})
        job_descriptions = soup.find_all("div", {"class": "jb-descr m10t t-small"})
        posted = soup.find_all("div", {"class": "jb-date col p0x p0t t-mute"})

        company_names = [span.get_text(strip=True) for span in company__names]
        base_url = "https://www.bayt.com"

        for i in range(len(job_titles)):
            try:
                link_tag = job_titles[i].find("a")
                if link_tag and 'href' in link_tag.attrs:
                    href = link_tag.attrs['href']
                    if href.startswith("/"):
                        href = base_url + href
                else:
                    href = None
                jobs.append({
                    "job_title": job_titles[i].text.strip(),
                    "company_name": company_names[i],
                    "location": locations_names[i].text.strip(),
                    "description": job_descriptions[i].text.strip(),
                    "link": href, 
                    "date": posted[i].text.strip(),
                    "source": "bayt"
                })
            except AttributeError:
                print(f"Erreur lors de l'extraction des données d'une offre d'emploi sur {url}")
        return jobs
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors du scraping de {url} (Bayt) : {e}")
        return []

# Exemple d'utilisation
#for i in range(1, 3):
#    url = f"https://www.bayt.com/en/tunisia/jobs/ing%C3%A9nieur-informatique-jobs/?page={i}.html"
#    headers = {
#        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
#    }
#    jobs = scrape_bayt_page(url, headers)

    # Afficher les résultats
#    for job in jobs:
#        print(job)

# Liste des fichiers à exporter
job_titles = []
company_names = []
locations_names = []
job_descriptions = []
links = []
dates = []

# Exemple d'utilisation : boucler sur plusieurs pages pour récupérer les données
for i in range(1, 14):
    url = f"https://www.bayt.com/en/uae/jobs/ing%C3%A9nieur-jobs-in-dubai/?page={i}.html"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    jobs = scrape_bayt_page(url, headers)

    # Ajouter les données aux listes
    for job in jobs:
        job_titles.append(job["job_title"])
        company_names.append(job["company_name"])
        locations_names.append(job["location"])
        job_descriptions.append(job["description"])
        links.append(job["link"])
        dates.append(job["date"])

# Exporter les données en fichier CSV
file_list = [job_titles, company_names, locations_names, job_descriptions, links, dates]
exported = zip_longest(*file_list)  # Utilisation de zip_longest pour garantir l'alignement des colonnes

with open("/Users/Pc/Documents/dubai_jobs.csv", "w", newline='', encoding='utf-8') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(["job title", "company name", "location", "description", "links", "date"])  # Entêtes du CSV
    wr.writerows(exported)  # Écriture des données

# Partie 2 : Nettoyage et sauvegarde avec pandas
df = pd.read_csv('/Users/Pc/Documents/dubai_jobs.csv', encoding='ISO-8859-1', skip_blank_lines=True)

# Supprimer les lignes où toutes les valeurs sont manquantes, si nécessaire
#df = df.dropna(how='all')

# Sauvegarder le fichier nettoyé sans lignes vides
#df.to_csv('/Users/Pc/Documents/dubai_jobs_cleaned.csv', index=False)

print("Le fichier a été nettoyé et sauvegardé sans lignes vides sous 'dubai_jobs_cleaned.csv'.")

##########
import pandas as pd
from pymongo import MongoClient

# Connexion à MongoDB
client = MongoClient("mongodb://localhost:27017/")  # Changez si vous utilisez MongoDB Atlas

# Sélection de la base de données et de la collection
db = client["jobs"]
collection = db["job_posts"]

#Charger le fichier CSV avec pandas
df = pd.read_csv('/Users/Pc/Documents/dubai_jobs.csv')
#df = pd.read_csv("dubai_jobs.csv")
#Conversion du DataFrame en dictionnaire pour MongoDB
data = df.to_dict(orient="records")

# Insérer les données dans la collection MongoDB
collection.insert_many(data)

print("Importation terminée avec succès !")

Le fichier a été nettoyé et sauvegardé sans lignes vides sous 'dubai_jobs_cleaned.csv'.


FileNotFoundError: [Errno 2] No such file or directory: 'dubai_jobs_cleaned.csv'